In [1]:
GLOBAL_SEED = 7532

from numpy.random import seed
seed(GLOBAL_SEED)
from tensorflow import set_random_seed
set_random_seed(GLOBAL_SEED)

import numpy as np
import pandas as pd

from keras.models import Model, load_model, Sequential
from keras.callbacks import Callback, EarlyStopping, History, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Dense, Dropout, Conv2D, Flatten, Input, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50, preprocess_input

from sklearn.utils.class_weight import compute_class_weight

Using TensorFlow backend.


Read in the data.

In [2]:
train_set = pd.read_csv('train_set_metadata_HMGD.csv')
valid_set = pd.read_csv('valid_set_metadata_HMGD.csv')
test_set = pd.read_csv('test_set_metadata_HMGD.csv')

Initialize constants.

In [3]:
INPUT_SHAPE = (320, 320, 3)

LEARNING_RATE = 0.0002
N_EPOCHS = 50
BATCH_SIZE = 32

Define the CNN model architecture.

In [4]:
resnet50_model = ResNet50(weights='imagenet', include_top=False, pooling=None, input_shape=INPUT_SHAPE)
resnet50_model.summary()

c:\users\rafal_pikula_01\appdata\local\programs\python\python36\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 326, 326, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 160, 160, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 160, 160, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Freeze the convolutional layers, then create the model top and attach it to the model.

In [5]:
for layer in resnet50_model.layers:
    layer.trainable = False

In [7]:
input_shape = resnet50_model.outputs[0].get_shape().as_list()[1:]

max_pool  = MaxPool2D(2)(resnet50_model.layers[-1].output)
batch_nrm = BatchNormalization()(max_pool)

flatten   = Flatten(input_shape=input_shape)(batch_nrm)
dropout_1 = Dropout(rate=0.5, seed=GLOBAL_SEED)(flatten)
dense_1   = Dense(512, activation='elu')(dropout_1)
dropout_2 = Dropout(rate=0.5, seed=GLOBAL_SEED)(dense_1)
dense_2   = Dense(256, activation='elu')(dropout_2)
dropout_3 = Dropout(rate=0.5, seed=GLOBAL_SEED)(dense_2)
model_top = Dense(1, activation='sigmoid', name='top_output')(dropout_3)

model = Model(inputs=resnet50_model.inputs, outputs=model_top)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 326, 326, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 160, 160, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 160, 160, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Compute the class weights to be used during model training in order to mitigate the class imbalances.

In [4]:
class_values = train_set['gender'].values
classes = np.unique(class_values)

weights = compute_class_weight(
    'balanced', 
    classes, 
    class_values
)
class_weights = dict(zip(classes, weights))
class_weights

{0: 1.0949726587842759, 1: 0.9201873755187876}

Train the model (via transfer learning). Note that this initial training is manually stopped after a certain number of epochs.

In [9]:
# Create and compile the model
adam = Adam(
    lr=LEARNING_RATE
)
model.compile(
    optimizer=adam, 
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

# Initialize callbacks
checkpoint = ModelCheckpoint(
    'model_ResNet50_epoch_{epoch:03d}.hdf5', 
    monitor='val_loss', 
    save_best_only=False, 
    save_weights_only=False
)
lr_reduction = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=4
)
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10
)    
callback_list = [checkpoint, lr_reduction, early_stopping]

# generate and augment training and validation data
data_generator = ImageDataGenerator(
    rotation_range=20, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)
train_data_generator = data_generator.flow_from_dataframe(
    dataframe=train_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=BATCH_SIZE,
    seed=GLOBAL_SEED
)
valid_data_generator = data_generator.flow_from_dataframe(
    dataframe=valid_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=BATCH_SIZE,
    seed=GLOBAL_SEED
)

train_steps_per_epoch = train_data_generator.n//train_data_generator.batch_size
valid_steps_per_epoch = valid_data_generator.n//valid_data_generator.batch_size

#fit the model
history = model.fit_generator(
    generator=train_data_generator,
    steps_per_epoch=train_steps_per_epoch, 
    epochs=N_EPOCHS,
    callbacks=callback_list,
    validation_data=valid_data_generator,
    validation_steps=valid_steps_per_epoch,
    class_weight=class_weights,
    workers=4, 
    verbose=1
)

Found 165400 images.
Found 4176 images.
Epoch 1/50
5168/5168 [==============================] - 3593s 695ms/step - loss: 0.4486 - binary_accuracy: 0.8197 - val_loss: 0.3670 - val_binary_accuracy: 0.8377
Epoch 2/50
5168/5168 [==============================] - 3294s 637ms/step - loss: 0.3384 - binary_accuracy: 0.8567 - val_loss: 0.3713 - val_binary_accuracy: 0.8390
Epoch 3/50
5168/5168 [==============================] - 3301s 639ms/step - loss: 0.3238 - binary_accuracy: 0.8653 - val_loss: 0.3668 - val_binary_accuracy: 0.8419
Epoch 4/50
5168/5168 [==============================] - 3311s 641ms/step - loss: 0.3190 - binary_accuracy: 0.8670 - val_loss: 0.3749 - val_binary_accuracy: 0.8398
Epoch 5/50
5168/5168 [==============================] - 3306s 640ms/step - loss: 0.3148 - binary_accuracy: 0.8689 - val_loss: 0.3623 - val_binary_accuracy: 0.8528
Epoch 6/50
5168/5168 [==============================] - 3303s 639ms/step - loss: 0.3116 - binary_accuracy: 0.8699 - val_loss: 0.3808 - val_binary

KeyboardInterrupt: 

Fine-tune the model.

In [5]:
model = load_model('model_ResNet50_epoch_007.hdf5')

for layer in model.layers[-49:]:
    layer.trainable = True

# compile the model to reflect the above changes
adam = Adam(
    lr=0.0001
)
model.compile(
    optimizer=adam, 
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 326, 326, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 160, 160, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 160, 160, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [6]:
# Initialize callbacks
checkpoint = ModelCheckpoint(
    'model_ResNet50_finetuning_epoch_{epoch:03d}.hdf5', 
    monitor='val_loss', 
    save_best_only=False, 
    save_weights_only=False
)
lr_reduction = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=4
)
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10
)    
callback_list = [checkpoint, lr_reduction, early_stopping]

# generate and augment training and validation data
data_generator = ImageDataGenerator(
    rotation_range=20, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)
train_data_generator = data_generator.flow_from_dataframe(
    dataframe=train_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=BATCH_SIZE,
    seed=GLOBAL_SEED
)
valid_data_generator = data_generator.flow_from_dataframe(
    dataframe=valid_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=BATCH_SIZE,
    seed=GLOBAL_SEED
)

train_steps_per_epoch = train_data_generator.n//train_data_generator.batch_size
valid_steps_per_epoch = valid_data_generator.n//valid_data_generator.batch_size

#fit the model
history = model.fit_generator(
    generator=train_data_generator,
    steps_per_epoch=train_steps_per_epoch, 
    epochs=N_EPOCHS,
    callbacks=callback_list,
    validation_data=valid_data_generator,
    validation_steps=valid_steps_per_epoch,
    class_weight=class_weights,
    workers=4, 
    verbose=1
)

Found 165400 images.
Found 4176 images.
Epoch 1/50
5168/5168 [==============================] - 3676s 711ms/step - loss: 0.2853 - binary_accuracy: 0.8865 - val_loss: 0.3146 - val_binary_accuracy: 0.8716
Epoch 2/50
5168/5168 [==============================] - 3832s 741ms/step - loss: 0.2641 - binary_accuracy: 0.8960 - val_loss: 0.2884 - val_binary_accuracy: 0.8827
Epoch 3/50
5168/5168 [==============================] - 3905s 756ms/step - loss: 0.2549 - binary_accuracy: 0.8996 - val_loss: 0.3119 - val_binary_accuracy: 0.8617
Epoch 4/50
5168/5168 [==============================] - 3716s 719ms/step - loss: 0.2504 - binary_accuracy: 0.9015 - val_loss: 0.2948 - val_binary_accuracy: 0.8871
Epoch 5/50
5168/5168 [==============================] - 4137s 801ms/step - loss: 0.2459 - binary_accuracy: 0.9038 - val_loss: 0.2791 - val_binary_accuracy: 0.8839
Epoch 6/50
5168/5168 [==============================] - 3961s 766ms/step - loss: 0.2415 - binary_accuracy: 0.9047 - val_loss: 0.2958 - val_binary

Evaluate the model on the test set.

In [7]:
evaluation_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_dataframe(
    dataframe=test_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=1,
    seed=GLOBAL_SEED,
    shuffle=False
)

best_model = load_model('model_ResNet50_finetuning_epoch_011.hdf5')
best_model.evaluate_generator(generator=evaluation_data_generator, steps=len(test_set))

Found 7621 images.


[0.30704405580751865, 0.8825613436556883]